In [1]:
import matplotlib.pyplot as plt
import numpy as np

from commpy.modulation import QAMModem

from optic.dsp import pulseShape, firFilter, decimate, symbolSync, resample
from optic.models import phaseNoise, KramersKronigRx, linFiberCh

from optic.tx import simpleWDMTx
from optic.core import parameters
from optic.equalization import edc, mimoAdaptEqualizer
from optic.carrierRecovery import cpr
from optic.metrics import fastBERcalc, monteCarloGMI, monteCarloMI, signal_power
from optic.plot import pconst

import scipy.constants as const

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

### Simulation of a single polarization optical signal transmission

In [5]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 64                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 600000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
#paramTx.alphaRRC = 0.91        # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 50      # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
#FO = 5e9 + paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12       # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate

### Transmission with machine learning-based phase recovery

In [7]:
FO_Values = np.arange(1e9, 6e9, 1e9)
Rolloff = np.arange(0.01, 0.99, 0.1)

BER_NN = np.zeros((FO_Values.size, Rolloff.size))
SER_NN = np.zeros((FO_Values.size, Rolloff.size))
GMI_NN = np.zeros((FO_Values.size, Rolloff.size))
MI_NN  = np.zeros((FO_Values.size, Rolloff.size))
SNR_NN = np.zeros((FO_Values.size, Rolloff.size))

iterations = 5 # number of iterations per simulation
N = 256  # number of input amplitude samples to the NN

for indfile in range(0, iterations):
    for indNN, FO_ in enumerate(FO_Values):
        for index, indRoll_ in enumerate(Rolloff):
        
            # Main simulation parameters
            paramTx.alphaRRC = indRoll_
            FO = FO_ + paramTx.Rs/2
            # get the model according to the simulation parameters
            modelNN = tf.keras.models.load_model('Legacy_NN_models/testModel_SpS_'+str(paramADC.SpS_out)+'_FO_'+str(FO/10e9)+'GHz_Rolloff_'+str(round(paramTx.alphaRRC, 2)))
            
            # generate optical signal signal
            sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

            # simulate linear signal propagation
            sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

            # receiver detection and demodulation
            Fc = paramTx.Fc
            Ts = 1 / Fs
            mod = QAMModem(m=paramTx.M)

            freqGrid = paramTx.freqGrid
            print(
                "Demodulating channel #%d , fc: %.4f THz, λ: %.4f nm\n"
                % (
                    chIndex,
                    (Fc + freqGrid[chIndex]) / 1e12,
                    const.c / (Fc + freqGrid[chIndex]) / 1e-9,
                )
            )

            symbTx = symbTx_[:, :, chIndex]

            Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

            print(
                "Local oscillator P: %.2f dBm, lw: %.2f kHz, FO: %.2f MHz"
                % (Plo_dBm, lw / 1e3, FO / 1e6)
            )

            # generate LO field
            π = np.pi
            t = np.arange(0, len(sigCh))*Ts
            ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

            sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

            # Add LO to the received signal
            sigRx = np.sqrt(Plo) + sigCh * np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
            sfm = sigRx.copy()

            print('CSPR = %.2f dB' % (10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

            # ### Phase-retrieval stage

            # In[4]:

            # resampling to ADC sampling rate
            sigCh = resample(sigCh, paramADC)
            sfm = resample(sfm, paramADC)
            newFs = paramADC.SpS_out*paramTx.Rs

            # simulate ideal direct-detection optical receiver
            Amp = np.abs(sigRx)
            Amp = resample(Amp, paramADC)

            sfm = sfm/np.sqrt(signal_power(sfm))

            sigPhase = np.angle(sfm)  # get signal phase samples (labels) (L,)
            sigAmp   = np.pad(np.abs(sfm), (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)

            # create set of input features
            X_input = np.zeros((len(sigPhase), N))  # (L,N)

            for indPhase in range(len(sigPhase)):
                X_input[indPhase] = sigAmp[indPhase:N+indPhase]

            phiTime_NN = modelNN.predict(X_input)
            phiTime_NN = phiTime_NN.reshape(-1,)

            # optical field reconstruction
            sigRx = np.abs(sfm)*np.exp(1j*phiTime_NN)

            # remove DC level
            sigRx -= np.mean(sigRx)  # np.sqrt(Plo)

            # downshift to baseband
            t = np.arange(0, len(sigRx))*(1/newFs)
            sigRx *= np.exp(-1j * (2 * π * FO * t))

            # correct for (possible) phase ambiguity
            rot = np.mean(sigCh/sigRx)
            sigRx = rot * sigRx
            sigRx = sigRx / np.sqrt(signal_power(sigRx))

            # #### Standard receiver processing

            # In[5]:

            # resample to 2 samples/symbol:
            paramRes = parameters()
            paramRes.Rs = paramTx.Rs
            paramRes.SpS_in = paramADC.SpS_out
            paramRes.SpS_out = 2

            sigRx = resample(sigRx, paramRes)

            # Matched filtering and CD compensation

            # Matched filtering
            if paramTx.pulse == "nrz":
                pulse = pulseShape("nrz", paramADC.SpS_out)
            elif paramTx.pulse == "rrc":
                pulse = pulseShape(
                    "rrc", paramRes.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs
                )

            pulse = pulse / np.max(np.abs(pulse))
            sigRx = firFilter(pulse, sigRx)

            # plot constellations after matched filtering
            #pconst(sigRx[0::paramRes.SpS_out], lim=True, R=3)

            # CD compensation
            sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

            #plot constellations after CD compensation
            #pconst(sigRx[0::paramRes.SpS_out], lim=True, R=2)

            # Downsampling to 2 sps and re-synchronization with transmitted sequences
            sigRx = sigRx.reshape(-1, 1)

            symbRx = symbolSync(sigRx, symbTx, 2)

            # In[6]:

            # Power normalization
            x = sigRx
            d = symbRx

            x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
            d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

            # Adaptive equalization
            mod = QAMModem(m=paramTx.M)

            paramEq = parameters()
            paramEq.nTaps = 15
            paramEq.SpS = 2
            paramEq.mu = [5e-3, 2e-3]
            paramEq.numIter = 5
            paramEq.storeCoeff = False
            paramEq.alg = ["nlms", "dd-lms"]
            paramEq.M = paramTx.M
            paramEq.L = [20000, 80000]

            y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

            discard = int(paramEq.L[0]/2)

            #plot constellations after adaptive equalization
            #pconst(y_EQ[discard:-discard, :], lim=True)

            # In[7]:

            # Carrier phase recovery
            paramCPR = parameters()
            paramCPR.alg = "bps"
            paramCPR.M = paramTx.M
            paramCPR.N = 75
            paramCPR.B = 64
            paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

            y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            discard = 5000

            # plot constellations after CPR
            #pconst(y_CPR[discard:-discard, :], lim=True)

            # #### Evaluate transmission metrics

            # In[8]:

            # correct for (possible) phase ambiguity
            for k in range(y_CPR.shape[1]):
                rot = np.mean(d[:, k] / y_CPR[:, k])
                y_CPR[:, k] = rot * y_CPR[:, k]

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            ind = np.arange(discard, d.shape[0] - discard)
            BER_NN[indNN][index], SER_NN[indNN][index], SNR_NN[indNN][index] = fastBERcalc(y_CPR[ind, :], d[ind, :], mod)
            GMI_NN[indNN][index], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :], mod)
            MI_NN[indNN][index] = monteCarloMI(y_CPR[ind, :], d[ind, :], mod)

            print("Results:\n")
            print("SER: %.2e" % (SER_NN[indNN][index]))
            print("BER: %.2e" % (BER_NN[indNN][index]))
            print("SNR: %.2f dB" % (SNR_NN[indNN][index]))
            print("MI: %.2f bits" % (MI_NN[indNN][index]))
            print("GMI: %.2f bits" % (GMI_NN[indNN][index]))

    np.save(r'/home/silas/Documentos/GitHub/PIVIC-Comunicacoes-Opticas/resultados/Data/BER_NN_'+str(indfile), BER_NN)

channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB
12500/12500 [==============================] - 16s 1ms/step


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.022443.
nlms pre-convergence training iteration #1
nlms MSE = 0.002968.
nlms pre-convergence training iteration #2
nlms MSE = 0.002542.
nlms pre-convergence training iteration #3
nlms MSE = 0.002305.
nlms pre-convergence training iteration #4
nlms MSE = 0.002150.


dd-lms - training stage #1
dd-lms MSE = 0.001042.
Results:

SER: 1.89e-04
BER: 3.15e-05
SNR: 29.58 dB
MI: 5.99 bits
GMI: 5.99 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB
12500/12500 [==============================] - 15s 1ms/step


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.020744.
nlms pre-convergence training iteration #1
nlms MSE = 0.003021.
nlms pre-convergence training iteration #2
nlms MSE = 0.002514.
nlms pre-convergence training iteration #3
nlms MSE = 0.002246.
nlms pre-convergence training iteration #4
nlms MSE = 0.002083.


dd-lms - training stage #1
dd-lms MSE = 0.001122.
Results:

SER: 3.33e-05
BER: 5.56e-06
SNR: 29.39 dB
MI: 6.00 bits
GMI: 6.00 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: 0.00 dBm

total WDM dignal power: 0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB
12500/12500 [==============================] - 15s 1ms/step


nlms - training stage #0


...

nlms pre-convergence training iteration #0
nlms MSE = 0.015723.
nlms pre-convergence training iteration #1
nlms MSE = 0.004345.
nlms pre-convergence training iteration #2
nlms MSE = 0.003877.
nlms pre-convergence training iteration #3
nlms MSE = 0.003527.
nlms pre-convergence training iteration #4
nlms MSE = 0.003263.


dd-lms - training stage #1
dd-lms MSE = 0.002085.
Results:

SER: 3.67e-04
BER: 6.11e-05
SNR: 26.66 dB
MI: 6.01 bits
GMI: 6.00 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: 0.00 dBm

total WDM dignal power: 0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 21000.00 MHz
CSPR = 12.00 dB
12500/12500 [==============================] - 16s 1ms/step


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.019482.
nlms pre-convergence training iteration #1
nlms MSE = 0.006744.
nlms pre-convergence training iteration #2
nlms MSE = 0.005480.
nlms pre-convergence training iteration #3
nlms MSE = 0.004659.
nlms pre-convergence training iteration #4
nlms MSE = 0.004116.


dd-lms - training stage #1
dd-lms MSE = 0.002814.
Results:

SER: 1.51e-03
BER: 2.56e-04
SNR: 25.26 dB
MI: 5.99 bits
GMI: 5.99 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: 0.00 dBm

total WDM dignal power: 0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 21000.00 MHz
CSPR = 12.00 dB
12500/12500 [==============================] - 16s 1ms/step


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.021498.
nlms pre-convergence training iteration #1
nlms MSE = 0.008211.
nlms pre-convergence training iteration #2
nlms MSE = 0.006524.
nlms pre-convergence training iteration #3
nlms MSE = 0.005545.
nlms pre-convergence training iteration #4
nlms MSE = 0.004955.


dd-lms - training stage #1
dd-lms MSE = 0.003451.
Results:

SER: 2.57e-03
BER: 4.31e-04
SNR: 24.41 dB
MI: 5.98 bits
GMI: 5.99 bits


#### Transmission with phase recovery using the KK algorithm

In [2]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 64                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 600000         # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
#paramTx.alphaRRC = 0.91        # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 50      # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16          # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
#FO = 5e9 + paramTx.Rs/2  # frequency offset
lw = 0*200e3       # linewidth
ϕ_lo = 0           # initial phase in rad
Plo_dBm = 12        # power in dBm

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate

In [3]:
FO_Values = np.arange(1e9, 6e9, 1e9)
Rolloff = np.arange(0.01, 0.99, 0.1)

BER_KK = np.zeros((FO_Values.size, Rolloff.size))
SER_KK = np.zeros((FO_Values.size, Rolloff.size))
GMI_KK = np.zeros((FO_Values.size, Rolloff.size))
MI_KK  = np.zeros((FO_Values.size, Rolloff.size))
SNR_KK = np.zeros((FO_Values.size, Rolloff.size))

iterations = 5 # number of iterations per simulation

for indfile in range(0, iterations):
    for indNN, FO_ in enumerate(FO_Values):
        for index, indRoll_ in enumerate(Rolloff):
            # Main simulation parameters
            paramTx.alphaRRC = indRoll_
            FO = FO_ + paramTx.Rs/2
            
            # generate optical signal signal
            sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

            # simulate linear signal propagation
            sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

            # receiver detection and demodulation
            Fc = paramTx.Fc
            Ts = 1 / Fs
            mod = QAMModem(m=paramTx.M)

            freqGrid = paramTx.freqGrid
            print(
                "Demodulating channel #%d , fc: %.4f THz, λ: %.4f nm\n"
                % (
                    chIndex,
                    (Fc + freqGrid[chIndex]) / 1e12,
                    const.c / (Fc + freqGrid[chIndex]) / 1e-9,
                )
            )

            symbTx = symbTx_[:, :, chIndex]

            Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

            print(
                "Local oscillator P: %.2f dBm, lw: %.2f kHz, FO: %.2f MHz"
                % (Plo_dBm, lw / 1e3, FO / 1e6)
            )

            # generate LO field
            π = np.pi
            t = np.arange(0, len(sigCh))*Ts
            ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

            sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

            # Add LO to the received signal
            sigRx = np.sqrt(Plo) + sigCh * np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
            sfm = sigRx.copy()

            print('CSPR = %.2f dB' % (10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

            # ### Phase-retrieval stage

            # In[4]:

            # resampling to ADC sampling rate
            sigCh = resample(sigCh, paramADC)
            sfm = resample(sfm, paramADC)
            newFs = paramADC.SpS_out*paramTx.Rs

            # simulate ideal direct-detection optical receiver
            Amp = np.abs(sigRx)
            Amp = resample(Amp, paramADC)

            # Kramers-Kronig phase-retrieval
            phiTime = KramersKronigRx(Amp, newFs)

            # optical field reconstruction
            sigRx = Amp*np.exp(1j*phiTime)

            # remove DC level
            sigRx -= np.sqrt(Plo)  # np.mean(sigRx)

            # downshift to baseband
            t = np.arange(0, len(sigRx))*(1/newFs)
            sigRx *= np.exp(-1j * (2 * π * FO * t))

            # correct for (possible) phase ambiguity
            rot = np.mean(sigCh/sigRx)
            sigRx = rot * sigRx
            sigRx = sigRx / np.sqrt(signal_power(sigRx))

            # #### Standard receiver processing

            # In[5]:

            # resample to 2 samples/symbol:
            paramRes = parameters()
            paramRes.Rs = paramTx.Rs
            paramRes.SpS_in = paramADC.SpS_out
            paramRes.SpS_out = 2

            sigRx = resample(sigRx, paramRes)

            # Matched filtering and CD compensation

            # Matched filtering
            if paramTx.pulse == "nrz":
                pulse = pulseShape("nrz", paramADC.SpS_out)
            elif paramTx.pulse == "rrc":
                pulse = pulseShape("rrc", paramRes.SpS_out, N=paramTx.Ntaps, alpha=paramTx.alphaRRC, Ts=1 / paramTx.Rs)

            pulse = pulse / np.max(np.abs(pulse))
            sigRx = firFilter(pulse, sigRx)

            # plot constellations after matched filtering
            #pconst(sigRx[0::paramRes.SpS_out], lim=True, R=3)

            # CD compensation
            sigRx = edc(sigRx, Ltotal, D, Fc, paramRes.SpS_out*paramTx.Rs)

            #plot constellations after CD compensation
            #pconst(sigRx[0::paramRes.SpS_out], lim=True, R=2)

            # Downsampling to 2 sps and re-synchronization with transmitted sequences
            sigRx = sigRx.reshape(-1, 1)

            symbRx = symbolSync(sigRx, symbTx, 2)

            # In[6]:

            # Power normalization
            x = sigRx
            d = symbRx

            x = x.reshape(len(x), 1) / np.sqrt(signal_power(x))
            d = d.reshape(len(d), 1) / np.sqrt(signal_power(d))

            # Adaptive equalization
            mod = QAMModem(m=paramTx.M)

            paramEq = parameters()
            paramEq.nTaps = 15
            paramEq.SpS = 2
            paramEq.mu = [5e-3, 2e-3]
            paramEq.numIter = 5
            paramEq.storeCoeff = False
            paramEq.alg = ["nlms", "dd-lms"]
            paramEq.M = paramTx.M
            paramEq.L = [20000, 80000]

            y_EQ, H, errSq, Hiter = mimoAdaptEqualizer(x, dx=d, paramEq=paramEq)

            discard = int(paramEq.L[0]/2)

            #plot constellations after adaptive equalization
            #pconst(y_EQ[discard:-discard, :], lim=True)

            # In[7]:

            # Carrier phase recovery
            paramCPR = parameters()
            paramCPR.alg = "bps"
            paramCPR.M = paramTx.M
            paramCPR.N = 75
            paramCPR.B = 64
            paramCPR.pilotInd = np.arange(0, len(y_EQ), 20)

            y_CPR, θ = cpr(y_EQ, symbTx=d, paramCPR=paramCPR)

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            discard = 5000

            # plot constellations after CPR
            #pconst(y_CPR[discard:-discard, :], lim=True)

            # #### Evaluate transmission metrics

            # In[8]:

            # correct for (possible) phase ambiguity
            for k in range(y_CPR.shape[1]):
                rot = np.mean(d[:, k] / y_CPR[:, k])
                y_CPR[:, k] = rot * y_CPR[:, k]

            y_CPR = y_CPR / np.sqrt(signal_power(y_CPR))

            ind = np.arange(discard, d.shape[0] - discard)
            BER_KK[indNN][index], SER_KK[indNN][index], SNR_KK[indNN][index] = fastBERcalc(y_CPR[ind, :], d[ind, :], mod)
            GMI_KK[indNN][index], _ = monteCarloGMI(y_CPR[ind, :], d[ind, :], mod)
            MI_KK[indNN][index] = monteCarloMI(y_CPR[ind, :], d[ind, :], mod)

            print("Results:\n")
            print("SER: %.2e" % (SER_KK[indNN][index]))
            print("BER: %.2e" % (BER_KK[indNN][index]))
            print("SNR: %.2f dB" % (SNR_KK[indNN][index]))
            print("MI: %.2f bits" % (MI_KK[indNN][index]))
            print("GMI: %.2f bits" % (GMI_KK[indNN][index]))

    np.save(r'/home/silas/Documentos/GitHub/PIVIC-Comunicacoes-Opticas/resultados/Data/BER_KK_'+str(indfile), BER_KK)

channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.022294.
nlms pre-convergence training iteration #1
nlms MSE = 0.002431.
nlms pre-convergence training iteration #2
nlms MSE = 0.002025.
nlms pre-convergence training iteration #3
nlms MSE = 0.001795.
nlms pre-convergence training iteration #4
nlms MSE = 0.001643.


dd-lms - training stage #1
dd-lms MSE = 0.000985.
Results:

SER: 4.00e-04
BER: 6.85e-05
SNR: 29.87 dB
MI: 5.99 bits
GMI: 5.99 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.020253.
nlms pre-convergence training iteration #1
nlms MSE = 0.002711.
nlms pre-convergence training iteration #2
nlms MSE = 0.002244.
nlms pre-convergence training iteration #3
nlms MSE = 0.002001.
nlms pre-convergence training iteration #4
nlms MSE = 0.001852.


dd-lms - training stage #1
dd-lms MSE = 0.001181.
Results:

SER: 3.00e-04
BER: 5.00e-05
SNR: 29.24 dB
MI: 5.99 bits
GMI: 5.99 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: 0.00 dBm

total WDM dignal power: 0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 17000.00 MHz
CSPR = 12.00 dB


nlms - training stage #0


...

nlms pre-convergence training iteration #0
nlms MSE = 0.020154.
nlms pre-convergence training iteration #1
nlms MSE = 0.007322.
nlms pre-convergence training iteration #2
nlms MSE = 0.006040.
nlms pre-convergence training iteration #3
nlms MSE = 0.005314.
nlms pre-convergence training iteration #4
nlms MSE = 0.004883.


dd-lms - training stage #1
dd-lms MSE = 0.003573.
Results:

SER: 4.90e-03
BER: 8.22e-04
SNR: 24.31 dB
MI: 5.97 bits
GMI: 5.97 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 21000.00 MHz
CSPR = 12.00 dB


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.023220.
nlms pre-convergence training iteration #1
nlms MSE = 0.009034.
nlms pre-convergence training iteration #2
nlms MSE = 0.007117.
nlms pre-convergence training iteration #3
nlms MSE = 0.006210.
nlms pre-convergence training iteration #4
nlms MSE = 0.005733.


dd-lms - training stage #1
dd-lms MSE = 0.004449.
Results:

SER: 8.68e-03
BER: 1.47e-03
SNR: 23.36 dB
MI: 5.95 bits
GMI: 5.95 bits
channel 0	 fc : 193.1000 THz
  mode #0	 power: 0.00 dBm
channel 0	 power: -0.00 dBm

total WDM dignal power: -0.00 dBm
Demodulating channel #0 , fc: 193.1000 THz, λ: 1552.5244 nm

Local oscillator P: 12.00 dBm, lw: 0.00 kHz, FO: 21000.00 MHz
CSPR = 12.00 dB


nlms - training stage #0


  0%|          | 0/5 [00:00<?, ?it/s]

nlms pre-convergence training iteration #0
nlms MSE = 0.027559.
nlms pre-convergence training iteration #1
nlms MSE = 0.011225.
nlms pre-convergence training iteration #2
nlms MSE = 0.008821.
nlms pre-convergence training iteration #3
nlms MSE = 0.007810.
nlms pre-convergence training iteration #4
nlms MSE = 0.007280.


dd-lms - training stage #1
dd-lms MSE = 0.005454.
Results:

SER: 1.56e-02
BER: 2.65e-03
SNR: 22.40 dB
MI: 5.92 bits
GMI: 5.92 bits
